In [2]:
import os
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
import xarray as xr
from multiprocessing import Pool
from tqdm import tqdm
from joblib import Parallel, delayed

In [3]:
# Configuration
time_interval_reconstruction = ["2018-01-01 00:00:00", "2019-12-31 23:54:00"]
predict_variable = 'WSPDchyv2_2011_2019'
k = 3
folder_path = '/Users/Murilo/weather_data'
netCDF_resolution = 1

In [4]:
all_dataframes = {}  # This dictionary will store DataFrames for each nc file

nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]

for nc_file in nc_files:
    with xr.open_dataset(os.path.join(folder_path, nc_file)) as ds:
        data_dict = {}
        
        # Dynamically load all variables from the file
        for var_name, variable in ds.data_vars.items():
            data = variable.values
            
            data = np.where(data == 9.969210e+36, np.nan, data)
            fraction_available = np.sum(~np.isnan(data)) / data.size

            # If more than 80% of the data is available, store it in the dictionary
            if fraction_available > 0.80:
                data_dict[var_name] = data

        # Convert the time variable
        if 'time' in ds.coords:
            time_data = ds['time']   
            
            # Add time data to the dictionary
            data_dict['time'] = time_data
        
        # Convert the data dictionary to a DataFrame
        df = pd.DataFrame(data_dict)
        
        # Store this DataFrame in our all_dataframes dictionary
        all_dataframes[nc_file] = df

all_dataframes



{'chyv2_2011_2019.nc':         WSPD   WDIR  GST  ATMP                time
 0        2.3  210.0  3.6   8.3 2011-01-01 00:00:00
 1        2.4  197.0  3.6   8.3 2011-01-01 00:06:00
 2        2.7  190.0  3.7   8.1 2011-01-01 00:12:00
 3        2.6  186.0  3.5   8.0 2011-01-01 00:18:00
 4        2.6  190.0  3.9   7.8 2011-01-01 00:24:00
 ...      ...    ...  ...   ...                 ...
 788875   2.9  229.0  4.6  11.1 2019-12-31 23:30:00
 788876   2.9  225.0  4.2  11.0 2019-12-31 23:36:00
 788877   3.0  224.0  4.8  11.0 2019-12-31 23:42:00
 788878   2.9  226.0  3.9  10.9 2019-12-31 23:48:00
 788879   3.1  229.0  4.3  10.9 2019-12-31 23:54:00
 
 [788880 rows x 5 columns],
 'cryv2_2011_2019.nc':         WSPD   WDIR  GST         PRES                time
 0        2.4  171.0  2.5  1024.000000 2011-01-01 00:00:00
 1        2.5  169.0  2.6  1024.099976 2011-01-01 00:06:00
 2        2.4  159.0  2.7  1024.000000 2011-01-01 00:12:00
 3        3.1  147.0  3.3  1024.099976 2011-01-01 00:18:00
 4     

In [5]:
merged_df = None

for filename, df in all_dataframes.items():
    # Skip the 'time' column for renaming
    rename_dict = {col: col + filename[:-3] if col != 'time' else col for col in df.columns}
    df = df.rename(columns=rename_dict)

    if merged_df is None:
        merged_df = df
    else:
        # Merge the DataFrame with the main merged_df on 'time'
        merged_df = pd.merge(merged_df, df, on='time', how='outer')

print(merged_df.head())

start_time = merged_df['time'].iloc[0]
end_time = merged_df['time'].iloc[-1]

print(f"Start Time: {start_time}")
print(f"End Time: {end_time}")

# Extract client times
client_start_time = pd.Timestamp(time_interval_reconstruction[0])
client_end_time = pd.Timestamp(time_interval_reconstruction[1])

# Check if client times fall within the dataframe's time range
if client_start_time < start_time or client_end_time > end_time:
    print("⚠️ WARNING ⚠️")
    print("The time interval provided is OUTSIDE the range of the available data.")
    print(f"Available data range: {start_time} to {end_time}")
    print(f"Interval: {client_start_time} to {client_end_time}")
    print("Please provide a valid time range.")
else:
    print("✅ SUCCESS ✅")
    print("The time interval provided is INSIDE the range of the available data.")



   WSPDchyv2_2011_2019  WDIRchyv2_2011_2019  GSTchyv2_2011_2019  \
0                  2.3                210.0                 3.6   
1                  2.4                197.0                 3.6   
2                  2.7                190.0                 3.7   
3                  2.6                186.0                 3.5   
4                  2.6                190.0                 3.9   

   ATMPchyv2_2011_2019                time  WSPDcryv2_2011_2019  \
0                  8.3 2011-01-01 00:00:00                  2.4   
1                  8.3 2011-01-01 00:06:00                  2.5   
2                  8.1 2011-01-01 00:12:00                  2.4   
3                  8.0 2011-01-01 00:18:00                  3.1   
4                  7.8 2011-01-01 00:24:00                  3.2   

   WDIRcryv2_2011_2019  GSTcryv2_2011_2019  PREScryv2_2011_2019  \
0                171.0                 2.5          1024.000000   
1                169.0                 2.6          1024.099

In [7]:
# Splitting into training and testing based on the client's time range
training_data = merged_df[(merged_df['time'] < client_start_time) | (merged_df['time'] > client_end_time)]
testing_data = merged_df[(merged_df['time'] >= client_start_time) & (merged_df['time'] <= client_end_time)]

# Make a copy of the original dataframes for later use
training_with_na = training_data.copy()
testing_with_na = testing_data.copy()

# Drop NaNs for training and testing datasets
training_na = training_data.dropna()
testing_na = testing_data.dropna()

# Define X_train and Y_train
X_train = training_na.drop(columns=['time', predict_variable])
Y_train = training_na[predict_variable]

# Define X_val and Y_val
X_val = testing_na.drop(columns=['time', predict_variable])
Y_val = testing_na[predict_variable]
merged_df.to_csv('merged_df.csv', index=False)

In [6]:
# 1. Combine training_with_na and testing_with_na to get full dataset
full_data_with_na = pd.concat([training_with_na, testing_with_na], axis=0)

# Define the PLS model
pls = PLSRegression(n_components=2)
pls.fit(X_train, Y_train)

# Fit the PLS model on training data
X_all = pd.concat([X_train, X_val], axis=0)

# 2. Transform the data
X_all_transformed = pls.transform(X_all)

# Convert transformed data to DataFrame for easier manipulation
X_all_transformed_df = pd.DataFrame(X_all_transformed, index=X_all.index)

# Merge with the original timestamps
result = pd.merge(full_data_with_na[['time']], X_all_transformed_df, left_index=True, right_index=True, how='left')

print(result)

                      time         0         1
0      2011-01-01 00:00:00       NaN       NaN
1      2011-01-01 00:06:00       NaN       NaN
2      2011-01-01 00:12:00       NaN       NaN
3      2011-01-01 00:18:00       NaN       NaN
4      2011-01-01 00:24:00       NaN       NaN
...                    ...       ...       ...
788875 2019-12-31 23:30:00 -0.350146 -3.791242
788876 2019-12-31 23:36:00 -0.692525 -3.770110
788877 2019-12-31 23:42:00 -0.741699 -3.608891
788878 2019-12-31 23:48:00 -0.738279 -3.726141
788879 2019-12-31 23:54:00 -0.777975 -3.658096

[788880 rows x 3 columns]


In [7]:
# Assuming the variables k and result are already defined
nb_LVs = result.shape[1]

fit_data = np.full((k, nb_LVs), np.nan)

LVs = np.vstack([fit_data, result, fit_data])

LVs_df = pd.DataFrame(LVs)
LVs_df.index = LVs_df[0]

# Using LVs_df instead of LVs
Y = []
for j in range(1, LVs_df.shape[1]):
    data_into_vectors = []
    for i in range(2 * k + 1):
        data_into_vectors.append(LVs_df.iloc[i:i + len(LVs_df) - 2*k, j].values)       
    Y.append(np.column_stack(data_into_vectors))

Y_all = np.column_stack(Y)
Y_all_df = pd.DataFrame(Y_all)

print(Y_all_df)

              0         1         2         3         4         5         6   \
0            NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1            NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2            NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3            NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4            NaN       NaN       NaN       NaN       NaN       NaN       NaN   
...          ...       ...       ...       ...       ...       ...       ...   
788875  -0.40107       NaN       NaN -0.350146 -0.692525 -0.741699 -0.738279   
788876       NaN       NaN -0.350146 -0.692525 -0.741699 -0.738279 -0.777975   
788877       NaN -0.350146 -0.692525 -0.741699 -0.738279 -0.777975       NaN   
788878 -0.350146 -0.692525 -0.741699 -0.738279 -0.777975       NaN       NaN   
788879 -0.692525 -0.741699 -0.738279 -0.777975       NaN       NaN       NaN   

              7         8         9    

In [8]:
assert merged_df.shape[0] == result.shape[0], "Dataframes have different sizes!"

# Use a coluna 'time' de 'merged_df' para criar um array booleano
time_conditions = (merged_df['time'] < client_start_time) | (merged_df['time'] > client_end_time)

Y_analogues = Y_all_df[time_conditions]
Y_pred = Y_all_df[~time_conditions]



In [ ]:
# Convert dataframes to numpy arrays for faster computation
Y_analogues_np = Y_analogues.values
Y_pred_np = Y_pred.values

# Function to compute the most similar row for a single row in Y_pred
def find_most_similar(row):
    distances = np.nansum((Y_analogues_np - row) ** 2, axis=1)
    return np.argmin(distances)

# Use joblib to parallelize the operation
n_jobs = 8  # This will use all CPU cores. Adjust as needed.
similar_rows = Parallel(n_jobs=n_jobs)(delayed(find_most_similar)(row) for row in tqdm(Y_pred_np))

Y_pred['most_similar_row_in_Y_analogues'] = similar_rows



  0%|▏                                                                         | 320/175200 [05:29<51:28:02,  1.06s/it]

In [ ]:
# Convert dataframes to PyTorch tensors and move to GPU
Y_analogues_tensor = torch.tensor(Y_analogues.values, dtype=torch.float32).cuda()
Y_pred_tensor = torch.tensor(Y_pred.values, dtype=torch.float32).cuda()

# Function to compute the most similar row for a single row in Y_pred using GPU
def find_most_similar_gpu(row):
    distances = torch.sum((Y_analogues_tensor - row) ** 2, axis=1)
    _, min_index = torch.min(distances, 0)
    return min_index.item()

# Compute most similar rows using GPU
similar_rows = [find_most_similar_gpu(row) for row in tqdm(Y_pred_tensor, desc="Finding Similar Rows")]

Y_pred['most_similar_row_in_Y_analogues'] = similar_rows



